In [1]:
# 必要なもののインポート

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from keras.datasets import mnist
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
# データ読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 正規化
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# onehotベクトル化
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

# 訓練データと評価データに
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

11501568/11490434 [==============================] - 0s 0us/step


クラス、活性化関数、初期化、最適化手法、ミニバッチ処理などのクラスを定義  
（今までのものを流用）

In [57]:
class FC:
    """
    Fully connected layers from number of nodes n_nodes1 to n_nodes2
    Parameters
    ----------
    n_nodes1 : int
      Number of nodes in the previous layer
    n_nodes2 : int
      Number of nodes in subsequent layers
    initializer : Instances of initialization methods
    optimizer : Instances of optimization methods
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer, activation):
        
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation
        # Initialize.
        # Use the initializer method to initialize self.W and self.B
        self.W = self.initializer.W(self.n_nodes1,self.n_nodes2)
        self.B = self.initializer.B(self.n_nodes2)
        
    def forward(self, X):
        """
        Forward
        Parameters
        ----------
        X : ndarray of the following form, shape (batch_size, n_nodes1)
            Input
        Returns
        ----------
        A : ndarray of the following form, shape (batch_size, n_nodes2)
            Output
        """
        self.X = X
        self.A = np.dot(self.X,self.W) + self.B
        
        return self.activation.forward(self.A)
    
    def backward(self, dZ):
        """
        Backward
        Parameters
        ----------
        dA : ndarray of the following form, shape (batch_size, n_nodes2)
            The gradient flowed in from behind.
        Returns
        ----------
        dZ : ndarray of the following form, shape (batch_size, n_nodes1)
            forward slope
        """
        dA = self.activation.backward(dZ)
        self.dB = np.mean(dA,axis=0)
        self.dW = np.dot(self.X.T,dA)/len(self.X)
        dZ = np.dot(dA,self.W.T)
        
        # Update
        self = self.optimizer.update(self)
        
        return dZ
    
class SimpleInitializer:
    """
    Simple initialization with Gaussian distribution
    Parameters
    ----------
    sigma : float
      Standard deviation of Gaussian distribution
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        """
        Initializing weights
        Parameters
        ----------
        n_nodes1 : int
          Number of nodes in the previous layer
        n_nodes2 : int
          Number of nodes in subsequent layers

        Returns
        ----------
        W : weight
        """
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)
    
    def B(self, n_nodes2):
        """
        Bias initialization
        Parameters
        ----------
        n_nodes2 : int
          Number of nodes in subsequent layers

        Returns
        ----------
        B : bias
        """
        return np.zeros(n_nodes2)
    
class HeInitializer():
    """
    Initialization of weights by He
    """
    def __init__(self):
        pass
        
    def W(self, n_nodes1, n_nodes2):
        """
        Initializing weights
        Parameters
        ----------
        n_nodes1 : int
          Number of nodes in the previous layer
        n_nodes2 : int
          Number of nodes in subsequent layers

        Returns
        ----------
        W : weight
        """
        return np.random.randn(n_nodes1, n_nodes2)*np.sqrt(2/n_nodes1)
    
    def B(self, n_nodes2):
        """
        Bias initialization
        Parameters
        ----------
        n_nodes2 : int
          Number of nodes in subsequent layers

        Returns
        ----------
        B : bias
        """
        return np.zeros(n_nodes2)
    
class SGD:
    """
    stochastic gradient descent method
    Parameters
    ----------
    lr : learning rate
    """
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, layer):
        """
        Updating the weights and biases of a layer
        Parameters
        ----------
        layer : An instance of the layer before the update
        """
        layer.W -= self.lr*layer.dW
        layer.B -= self.lr*layer.dB
        
        return layer
    
class AdaGrad:
    """
    stochastic gradient descent method
    Parameters
    ----------
    lr : learning rate
    """
    def __init__(self, lr):
        self.lr = lr
        self.hW = 0
        self.hB = 0
        
    def update(self, layer):
        """
        Updating the weights and biases of a layer
        Parameters
        ----------
        layer : An instance of the layer before the update
        """
        self.hW += layer.dW*layer.dW
        self.hB = layer.dB*layer.dB
    
        layer.W -= self.lr*layer.dW/(np.sqrt(self.hW) +1e-7)
        layer.B -= self.lr*layer.dB/(np.sqrt(self.hB) +1e-7)
        
        return layer
    
class ReLU():
    """
    Activation function : ReLU function
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        self.A = A
        return np.maximum(self.A,0)
    
    def backward(self,dZ):
        
        return np.where(self.A>0,dZ,0)
    
class Softmax():
    """
    Activation Function : Softmax Function
    """
    def __init__(self):
        pass
        
    def forward(self,A):
        
        return np.exp(A-np.max(A))/np.sum(np.exp(A-np.max(A)),axis=1,keepdims=True)
    
    def backward(self,dZ):
        return dZ
    
# Mini-batch processing class
class GetMiniBatch:
    """
    Iterator to get the mini-batch

    Parameters
    ----------
    X : ndarray of the following form, shape (n_samples, n_features)
      Training data
    y : ndarray of the following form, shape (n_samples, 1)
      correct value
    batch_size : int
      Batch size
    seed : int
      Seeding random numbers in NumPy
    """
    def __init__(self, X, y, batch_size = 20, seed=None):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
        
    def __len__(self):
        return self._stop
    
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1] 
    
    def __iter__(self):
        self._counter = 0
        return self
    
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

【問題1】2次元畳み込み層の作成

In [4]:
# 畳み込み層の初期パラメータWとBの生成するクラスを作成

class CNN2d_Initializer:
    """
    畳み用のパラメータ初期値
    Wの引数は４つ、フィルタ数（出力チャンネル数、入力チャンネル数、フィルタの縦、フィルタの横）
    F_num
    inch_num
    FH
    FW
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, F_num, inch_num, FH, FW):
        """
        Initializing weights
        Parameters
        ----------
        n_nodes1 : int
          Number of nodes in the previous layer
        n_nodes2 : int
          Number of nodes in subsequent layers

        Returns
        ----------
        W : weight
        """
        return self.sigma * np.random.randn(F_num, inch_num, FH, FW)

    def B(self, F_num):
        """
        Bias initialization
        Parameters
        ----------
        n_nodes2 : int
          Number of nodes in subsequent layers

        Returns
        ----------
        B : bias
        """
        return self.sigma * np.random.randn(F_num)

In [5]:
# CNN2d_Initializerの動作確認
# 希望する配列の初期値が確認できた

a=CNN2d_Initializer(0.01)
print(a.W(2,2,4,5).shape)
print(a.B(2).shape)

(2, 2, 4, 5)
(2,)


In [6]:
# 出力を計算する関数を制作しておく
# パディングは左右上下対称、ストライドも幅も横と縦は対称とする

def output_2d(H, W, FH, FW, P=0, S=1):
  """
  Hは入力の縦、Wは入力の横
  """
  OH=(H+2*P-FH)/S+1
  OW=(W+2*P-FW)/S+1
  OH=int(OH)
  OW=int(OW)
  return OH,OW

In [7]:
#　動作確認
output_2d(7, 7, 3, 3, P=0, S=2)

(3, 3)

畳み込み層の雛形完成

In [8]:
class Conv2d:
    """
    フィルタ数（出力チャンネル、いくつのチャンネルで出力したいか）、入力チャンネル（入力に合わせる）
    フィルタの縦、フィルタの横、初期値クラス（上で作ったやつ）、最適化クラス、活性化関数クラス
    を引数とする
    """

    def __init__(self, F_num, ch_num, FH, FW, initializer, optimizer, activation):
        
        self.F_num=F_num
        self.ch_num=ch_num
        self.FH=FH
        self.FW=FW

        self.initializer = initializer
        self.optimizer = optimizer
        self.activation = activation

        self.W = self.initializer.W(F_num, ch_num, FH, FW)
        self.B = self.initializer.B(F_num)
        
    def forward(self, X):
        """
        """
        self.X = X
        #入力値の情報を取得　Xはバッチ数、チャンネル数、縦、横の４次元配列を想定
        N,C,H,W=X.shape
        #上で作ったアウトプット数関数でアウトプットの縦と横を取得
        OH,OW=output_2d(H, W, self.FH, self.FW, P=0, S=1)
        #フォワードプロパゲーションを行なった際のAの値を入れる箱を作る
        A=np.zeros([N,self.F_num,OH,OW])
        #パディング層は一旦保留、ストライドも１で、一般化は保留

        #アウトプットのチャンネルは１にしておく
        self.out_ch=1

        #ループによるフォワードプロパゲーション計算
        #計算してAに入れる
        for i in range(self.X.shape[0]):
          for j in range(self.W.shape[0]):
            for k in range(OH):
              for l in range(OW):
                total=0
                for m in range(self.W.shape[2]):
                  for n in range(self.W.shape[3]):
                    total+=self.X[i,self.out_ch-1,k+m,l+n]*self.W[j,self.out_ch-1,m,n]
                A[i,j,k,l]=total
        self.A=A
        return self.A
    
    def backward(self, dA):
        """
        Backward
        """
        #dAはでて行った配列が帰ってくるので　A=np.zeros([N,self.F_num,OH,OW])
        #逆伝播後の空箱を作る
        #パディング層は一旦保留、ストライドも１で、一般化は保留
        #dZ=np.zeros([N,C,H,W])
        #足し合わせるためにWとバッチ数を増やした、５次元配列にする、その後、バッチ数で足し合わせる
        #dW=np.zeros(N,self.F_num,self.ch_num,self.FH,self.FW)
        #dB=np.zeros(self.B.shape)
        #ループによるバックプロパゲーション計算
        #計算してdZ入れる
        self.out_ch=1

        dZ = np.zeros(self.X.shape)
        N,C,H,W = self.X.shape
        F,C,FH,FW = self.W.shape
        OH,OW=output_2d(H, W, self.FH, self.FW, P=0, S=1)

        for n in range(N):
          for i in range(H):
            for j in range(W):
              total=0
              for m in range(F):
                for s in range(FH):
                  for t in range(FW):
                    if (i-s<0 or i-s>OH-1or j-t<0 or j-t>OW-1) :
                      continue
                    total+=dA[n,m,i-s,j-t]*self.W[m,self.out_ch-1,s,t]
                dZ[n,self.out_ch-1,i,j]=total

        dw = np.zeros(self.W.shape)
        for k1 in range(F):
          for s1 in range(FH):
            for t1 in range(FW):
                total=0
                for n1 in range(N):
                  for i1 in range(OH):
                    for j1 in range(OW):
                      total+=dA[n1,k1,i1,j1]*self.X[n1,self.out_ch-1,i1+s1,j1+t1]
                  dw[k1,self.out_ch-1,s1,t1]=total
        self.dW=dw


        dB = np.zeros(self.W.shape[0])
        for i2 in range(dA.shape[1]):
          dB[i2]=np.sum(dA[:,i2])
        self.dB=dB

        # Update
        self = self.optimizer.update(self)
        
        return dZ

【問題2】小さな配列での2次元畳み込み層の実験

In [9]:
# データの定義
x = np.array([[[[ 1,  2,  3,  4],[ 5,  6,  7,  8],[ 9, 10, 11, 12],[13, 14, 15, 16]]]])
w = np.array([[[[ 0.,  0.,  0.],[ 0.,  1.,  0.],[ 0., -1.,  0.]]],[[[ 0.,  0.,  0.],[ 0., -1.,  1.],[ 0.,  0.,  0.]]]])
da = np.array([[[[ -4,  -4], [ 10,  11]],[[  1,  -7],[  1, -11]]]])

In [10]:
F_num=2
ch_num=1
FH=3
FW=3
initializer=CNN2d_Initializer(0.01)
optimizer=SGD(0.01)
activation=ReLU()

cnn1=Conv2d(F_num, ch_num, FH, FW, initializer, optimizer, activation)
cnn1.W = w

In [11]:
# フォワードプロパゲーションを実行
# 期待通りの結果がでた
cnn1.forward(x)

array([[[[-4., -4.],
         [-4., -4.]],

        [[ 1.,  1.],
         [ 1.,  1.]]]])

In [12]:
# フォワードプロパゲーションを実行
# 期待通りの結果がでた
dZ=cnn1.backward(da)
dZ = np.delete(dZ,[0,4-1],axis=2)
dZ = np.delete(dZ,[0,4-1],axis=3)
dZ

array([[[[-5.,  4.],
         [13., 27.]]]])

【問題3】2次元畳み込み後の出力サイズ  
すでに雛形制作の前段階で実装 

In [13]:
def output_2d(H, W, FH, FW, P=0, S=1):
  """
  Hは入力の縦、Wは入力の横
  """
  OH=(H+2*P-FH)/S+1
  OW=(W+2*P-FW)/S+1
  OH=int(OH)
  OW=int(OW)
  return OH,OW

In [14]:
#　動作確認
output_2d(7, 7, 3, 3, P=0, S=2)

(3, 3)

【問題4】最大プーリング層の作成

サンプルを自分で解釈し実行した

In [15]:
class MaxPool2D():
    """最大プーリング層
    """
    def __init__(self,P):
        """コンストラクタ
        Parameters
        -----------
        P : プーリング幅
        """
        self.P = P
        # 順伝播の返り値
        self.PA = None
        # 最大値のインデックス記録
        self.Pindex = None
        
    def forward(self,A):
        """順伝播
        Parameters
        -----------
        A : 入力配列
        """
        # 入力配列のサイズ
        N,F,OH,OW = A.shape
        # 
        PS = self.P
        # 縦軸と横軸のスライド回数
        PH,PW = int(OH/PS),int(OW/PS)
        
        # 各種パラメータの保存
        self.params = N,F,OH,OW,PS,PH,PW
        
        # プーリング処理のための初期化
        self.PA = np.zeros([N,F,PH,PW])
        self.Pindex = np.zeros([N,F,PH,PW])
        
        # バッチ数でループ
        for n in range(N):
            # フィルター数でループ
            for ch in range(F):
                # 縦方向スライド回数
                for row in range(PH):
                    # 横方向スライド回数
                    for col in range(PW):
                        # 順伝播の値計算
                        self.PA[n,ch,row,col] = \
                        np.max(A[n,ch,row*PS:row*PS+PS,col*PS:col*PS+PS])
                        # 最大値のインデックス記録
                        self.Pindex[n,ch,row,col] = \
                        np.argmax(A[n,ch,row*PS:row*PS+PS,col*PS:col*PS+PS])
                        
        return self.PA
    
    def backward(self,dA):
        """逆伝播の値
        Parameters
        -----------
        dA : 逆伝播してきた値
        """
        # 保存しておいた各種パラメータ取得
        N,F,OH,OW,PS,PH,PW = self.params
        # 逆伝播の値
        dP = np.zeros([N,F,OH,OW])
        # バッチ数でループ
        for n in range(N):
            # フィルター数でループ
            for ch in range(F):
                # 縦方向スライド回数
                for row in range(PH):
                    # 横方向スライド回数
                    for col in range(PW):
                        # 最大値を取得してきたインデックスの取得
                        idx = self.Pindex[n,ch,row,col]
                        # 逆伝播の一時保存変数
                        tmp = np.zeros((PS*PS))
                        for i in range(PS*PS):
                            # 該当インデックスはその値
                            if i == idx:
                                tmp[i] = dA[n,ch,row,col]
                            # それ以外は0
                            else:
                                tmp[i] = 0
                        # 返り値の該当場所に格納
                        dP[n,ch,row*PS:row*PS+PS,col*PS:col*PS+PS] = tmp.reshape(PS,PS)
        
        return dP

以下では自分での復習のためテスト実行を行う

In [16]:
# テスト
# データ準備
X1 = np.random.randint(0,9,36).reshape(1,1,6,6)

In [17]:
X1

array([[[[5, 1, 0, 7, 0, 3],
         [0, 3, 3, 6, 4, 8],
         [4, 7, 7, 3, 5, 6],
         [1, 8, 0, 1, 0, 7],
         [0, 5, 1, 2, 3, 7],
         [3, 3, 4, 0, 5, 6]]]])

In [19]:
Pooling = MaxPool2D(P=2)
A = Pooling.forward(X1)
A

array([[[[5., 7., 8.],
         [8., 7., 7.],
         [5., 4., 7.]]]])

In [20]:
# 逆伝播してきた配列定義
dA = np.random.randint(0,9,9).reshape(A.shape)
dA

array([[[[6, 2, 1],
         [0, 0, 7],
         [4, 7, 1]]]])

In [21]:
dZ = Pooling.backward(dA)
dZ

array([[[[6., 0., 0., 2., 0., 0.],
         [0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 7.],
         [0., 4., 0., 0., 0., 1.],
         [0., 0., 7., 0., 0., 0.]]]])

【問題6】平滑化

In [22]:
class Flatten:
    """平滑化レイヤー"""
    def __ini__(self):
        """コンストラクタ"""
        pass
    def forward(self,X):
        """順伝播
        Parameters
        -----------
        X : 入力配列
        """
        self.shape = X.shape
        return X.reshape(len(X),-1)

    def backward(self,X):
        """逆伝播の値
        Parameters
        -----------
        X : 逆伝播してきた値
        """
        return X.reshape(self.shape)

In [23]:
# テスト
# データ準備
TEST = np.zeros([20,2,5,5])

# インスタンス生成
flt = Flatten()

# 順伝播
flat_forward = flt.forward(TEST)

# 逆伝播
flat_back = flt.backward(flat_forward)

print('Forward_shape:',flat_forward.shape)
print('Backward_shape:',flat_back.shape)

Forward_shape: (20, 50)
Backward_shape: (20, 2, 5, 5)


【問題7】学習と推定  
DNNでの書き方でクラスを作成する  
そのためいくつかクラスを定義し直している

In [58]:
class Softmax:
    
    def forward(self, X):
        self.Z = np.exp(X) / np.sum(np.exp(X), axis=1).reshape(-1,1)
        return self.Z
    
    def backward(self, Y):
        self.loss = self.loss_func(Y)
        return self.Z - Y
    
    def loss_func(self, Y, Z=None):
        if Z is None:
            Z = self.Z
        return (-1)*np.average(np.sum(Y*np.log(Z), axis=1))

In [68]:
class FC:
    """全結合層"""
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        """コンストラクタ
        Parameters
        ----------
        n_nodes1 : 前の層のノード数
        n_nodes2 : 当該層のノード数
        initializer : 初期化インスタンス
        optimizer : 勾配更新手法
        """
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # 初期化インスタンスの関数実行
        self.W = initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = initializer.B(self.n_nodes2)
        # 最適化インスタンス
        self.optimizer = optimizer
        # 勾配更新の際に使用（AdaGradのみ）
        self.HW = 0
        self.HB = 0
        
    def forward(self, X):
        """順伝播
        Parameters
        ----------
        X : 順伝播されてきた値
        """
        # 逆伝播時に使用
        self.Z = X
        # 順伝播計算部分本体
        self.A = X @ self.W + self.B
        return self.A
    
    def backward(self, dA):
        """逆伝播
        Parameters
        ----------
        dA : 前の層から逆伝播してきた値（活性化関数の逆伝播の値が入ってくる）
        
        Overview
        ----------
        前回のSprint9 ニューラルネットワークでは下記のような逆伝播処理になっていた
            0 ## 2層目
            1 dZ2 = dA3 @ self.W3.T
            2 dA2 = dZ2 * (1 - self.tanh_function(self.A2)**2)
            3 dW2 = self.Z1.T @ dA2
            4 dB2 = np.sum(dA2, axis=0)
            5 ## 1層目
            6 dZ1 = dA2 @ self.W2.T
            7 dA1 = dZ1 * (1 - self.tanh_function(self.A1)**2)
            8 dW1 = X.T @ dA1
            9 dB1 = np.sum(dA1, axis=0)
        勾配の計算
            ここでは、活性化関数の逆伝播は別で実装し、その値をこの関数の引数として受け取っているので、
            この関数の  dA  は、Sprint9の上記の  dA2  に該当する
            よって、上記3,4に該当する処理を書いていけばいい
        逆伝播の値の計算
            活性化関数の逆伝播に渡してやる値、つまりSprint9の上記の  dZ1  に該当する
        重み更新
            勾配dB,dWが計算されているので、このインスタンス自身をoptimizerインスタンスのupdate関数に渡してやる
            optimizerインスタンスのupdate関数の引数は、layerとなっているので、update関数内では、layer.変数名で
            このインスタンスの各種メンバ変数にアクセスできる
        """
        # バイアス項の勾配
        self.dB = np.sum(dA, axis=0)
        # バイアス項以外の勾配
        self.dW = self.Z.T @ dA
        # 逆伝播させる値
        self.dZ = dA @ self.W.T
        # 重み更新
        self = self.optimizer.update(self)
        return self.dZ

In [73]:
class FC:

    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        
    def forward(self, X):
        self.X = X
        A = X@self.W + self.B
        return A
    
    def backward(self, dA):
        dZ = dA@self.W.T
        self.dB = np.sum(dA, axis=0)
        self.dW = self.X.T@dA
        self.optimizer.update(self)
        return dZ

In [85]:
class SimpleInitializer:

    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, *shape):
        W = self.sigma * np.random.randn(*shape)
        return W
    
    def B(self, *shape):
        B = self.sigma * np.random.randn(*shape)
        return B

In [92]:
class CNN_sample_class():

    def __init__(self,batch_size=10,n_nodes1 =169,n_nodes2 = 200,n_output =10,lr =0.01,epoch=10,sigma=0.02,optimizer=SGD, initializer=SimpleInitializer,activater=ReLU,output_activater=Softmax):
        """コンストラクタ
        Parameters
        ----------
        batch_size : バッチサイズ（default:20)
        n_features : 説明変数の数（default:784)
        n_nodes1 : 前の層のノード数（default:400)
        n_nodes2 : 当該層のノード数（default:200)
        n_output : 出力層のノード数（default:10)
        sigma : 初期化時のパラメータ（default:0.02)
        lr : 学習率（default:0.005)
        verbose : 計算過程の出力（default:True)
        epoch : 学習回数（default:10)
        optimizer : 最適化手法（default:SGD)
        initializer : 初期化方法（default:HeInitializer）
        activater : 活性化関数（default:ReLU）
        """
        self.batch_size = batch_size
        self.n_nodes1 = n_nodes1  
        self.n_nodes2 = n_nodes2 
        self.n_output = n_output
        self.lr = lr
        self.epoch = epoch
        self.optimizer = optimizer 
        self.sigma = sigma
        self.initializer = initializer 
        self.activater = activater
        self.output_activater = output_activater 
    
    def loss_function(self,y,yt):
        """クロスエントロピー誤差
        Parameters
        ----------
        y : 予測値
        yt : 正解データ
        """
        delta = 1e-7
        return -np.mean(yt*np.log(y+delta))
    
    def accuracy(self,Z,Y):
        """クロスエントロピー誤差
        Parameters
        ----------
        Z : 予測値
        Y : 正解データ
        """
        return accuracy_score(Y,Z)

    def fit(self, X, y, X_val=None, y_val=None):
        """学習
        Parameters
        ----------
        X : 訓練データの説明変数
        y : 訓練データの目的変数
        X_val : 評価データの説明変数
        y_val : 評価データの目的変数
        """
        # lossの記録用配列
        self.loss_train = [] 
        self.loss_val = [] 
        # 最適化手法の初期化
        optimizer = self.optimizer(self.lr)
        # 各層の初期化
        self.cnn=Conv2d(F_num=1, ch_num=1, FH=3, FW=3, initializer=CNN2d_Initializer(0.01), optimizer=SGD(0.01), activation=ReLU())
        self.activation1 = self.activater()

        self.FC2 = FC(self.n_nodes1, self.n_nodes2, self.initializer(self.sigma), optimizer)
        self.activation2 = self.activater()
        self.FC3 = FC(self.n_nodes2, self.n_output, self.initializer(self.sigma), optimizer)
        self.activation3 = self.output_activater()

        self.loss_epoch = [self.activation3.loss_func(y, self.forward_propagation(X))]
        
        # 学習回数分ループ
        for i in range(self.epoch):
            # ミニバッチイテレータ生成
            get_mini_batch = GetMiniBatch(X, y, batch_size=self.batch_size, seed=0)
            # ミニバッチイテレータループ
            for mini_X, mini_y in get_mini_batch:
                ## 順伝播
                # 1層目
                A1=self.cnn.forward(mini_X)
                B1 = self.activation1.forward(A1)
                Pooling = MaxPool2D(P=2)
                C1 = Pooling.forward(B1)
                flt = Flatten()
                Z1=flt.forward(C1)
                # 2層目
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                # 3層目
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                
                ## 逆伝播
                dA3 = self.activation3.backward(mini_y)
                dZ2 = self.FC3.backward(dA3)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.FC2.backward(dA2)

                dC1=flt.backward(dZ1)
                dB1 = Pooling.backward(dC1)
                dA1 = self.activation1.backward(dB1)
                dX1=self.cnn.backward(dA1) #dX1は使わない

            self.loss_epoch.append(self.activation3.loss_func(y, self.forward_propagation(X)))   

                # 評価データ見る
                #if X_val is not None:
                    ## 順伝播
                    # 1層目
                    #A1 = self.FC1.forward(X_val)
                    #Z1 = self.activation1.forward(A1)
                    # 2層目
                    #A2 = self.FC2.forward(Z1)
                    #Z2 = self.activation2.forward(A2)
                    # 3層目
                    #A3 = self.FC3.forward(Z2)
                    #Z3 = self.activation3.forward(A3)
                    # 損失計算と記録
                    #self.loss_val.append(self.activation3.backward(Z3, y_val)[1])

    def forward_propagation(self, X):
        """順伝播
        Parameters
        ----------
        X : 訓練データの説明変数
        """
        A1=self.cnn.forward(X)
        B1 = self.activation1.forward(A1)
        Pooling = MaxPool2D(P=2)
        C1 = Pooling.forward(B1)
        flt = Flatten()
        Z1=flt.forward(C1)
        # 2層目
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        # 3層目
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        return Z3
    
    def predict(self, X):
        """予測
        Parameters
        ----------
        X : 入力配列
        """
        A1=self.cnn.forward(X)
        B1 = self.activation1.forward(A1)
        Pooling = MaxPool2D(P=2)
        C1 = Pooling.forward(B1)
        flt = Flatten()
        Z1=flt.forward(C1)
        # 2層目
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        # 3層目
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)
        return np.argmax(Z3, axis=1)

In [96]:
# X_train_を４次元配列に変換
X_train_1=X_train_[:1000][:,np.newaxis,:,:]
y_train_1=y_train_[:1000]

In [97]:
CNN1=CNN_sample_class()

In [98]:
CNN1.fit(X_train_1,y_train_1)

In [99]:
CNN1.loss_epoch

[2.302732176046404,
 2.2959861450474475,
 2.293688051189238,
 2.291628383936008,
 2.2750057171847127,
 1.2210950738790662,
 0.5420295475416614,
 0.37878833756046687,
 0.2679796954365485,
 0.3825216218624811,
 0.16949145662318943]

In [109]:
#　精度は９１％となった
y_pred = CNN1.predict(X_test[:,np.newaxis,:,:][:100])
accuracy_score(y_test[:100], y_pred)

0.91

【問題10】出力サイズとパラメータ数の計算

1.  
入力サイズ : 144×144, 3チャンネル  
フィルタサイズ : 3×3, 6チャンネル  
ストライド : 1  
パディング : なし  
→ 出力サイズ：6×142×142  
→ パラメータ数（重み）（F×C×FH×FW）：162  
→ パラメータ数（バイアス）（F）：6

2.  
入力サイズ : 60×60, 24チャンネル  
フィルタサイズ : 3×3, 48チャンネル  
ストライド　: 1  
パディング : なし  
→ 出力サイズ：48×58×58  
→ パラメータ数（重み）（F×C×FH×FW）：10368  
→ パラメータ数（バイアス）（F）：48  

3.  
入力サイズ : 20×20, 10チャンネル  
フィルタサイズ: 3×3, 20チャンネル  
ストライド : 2  
パディング : なし  
→ 出力サイズ：20x9x9  
→ パラメータ数（重み）（F×C×FH×FW）：1800  
→ パラメータ数（バイアス）（F）：20